## Notebook Goal:  
Using existing NLP and LDA methodologies to perform topic modeling on docket texts. Three hyperparameters to consider:
1. to remove organization or not in docket texts, so organizations themselves won't become topics.
2. to remove names or not in docket texts, so names themselves won't become topics.
3. variations in topic numbers: [2, 3, 5, 10]

Will then perform visualizations and model summary output on every permutation/iteration.

In [99]:
import nltk
from nltk.tag.stanford import StanfordNERTagger
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords

In [302]:
from gensim.models.word2vec import LineSentence
from gensim.models import Phrases
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

#visualization libraries
import pyLDAvis
import pyLDAvis.gensim

In [26]:
import os
java_path = 'C:/Program Files/Java/jdk-10.0.1/bin/java.exe'
os.environ['JAVAHOME'] = java_path

In [39]:
#import corpus/docket texts from html to pandas DataFrame
def grab_dockets():
    files = []
    #get all .html files in the folder (all docket files are in .html)
    for file in os.listdir('docket_texts/'):
        if file.endswith('.html'):
            files.append(os.path.join('docket_texts/', file))

    df_docket_texts = pd.DataFrame()
    
    for i in range(len(files)): #gather all docket texts
    #for i in [0, 1]: #for testing purposes
        
        content = codecs.open(files[i], 'r', 'utf-8').read()
        #use beautiful soup to get the case ID
        soup = BeautifulSoup(content, 'lxml')
        case_id = str(soup.find_all('h3'))    
        bookmark1 = case_id.find('CASE #:') + len('CASE #:')
        bookmark2 = case_id.find('</h3>')
        case_id = case_id[bookmark1:bookmark2]

        #use pandas to grab tables in the html files
        docket_tables = pd.read_html(content)

        #error checking: gotta do this because there's different length of docket_list/
        #usually docket texts are in docket_list[3], but not always
        n = 0
        while docket_tables[n].isin(['Docket Text']).sum().sum() == 0:
            #print(n, docket_tables[n].isin(['Docket Text']).sum().sum())
            n += 1
                        
        #print(i, files[i])
        #print(docket_tables[n].head())

        #docket_tables[n] is the docket text table
        new_header = docket_tables[n].iloc[0]
        docket_tables[n] = docket_tables[n][1:]
        docket_tables[n].columns = new_header
        
        docket_tables[n]['#'] = pd.to_numeric(docket_tables[n]['#'],
                                              downcast = 'signed', errors = 'coerce')
        docket_tables[n]['Date Filed'] = pd.to_datetime(docket_tables[n]['Date Filed'])
        docket_tables[n]['Case ID'] = case_id

        df_docket_texts = pd.concat([df_docket_texts, docket_tables[n]])
    #reorder a column
    cols = list(df_docket_texts.columns)
    df_docket_texts = df_docket_texts[[cols[-1]] + cols[:-1]]
    
    print('current docket text table size/shape: {}'.format(df_docket_texts.shape))
    return df_docket_texts

In [40]:
df = grab_dockets()
docket_original = list(df['Docket Text'])
for i in range(5):
    print('docket text {}'.format(i))
    print(docket_original[i], '\n')

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  self.parser.feed(markup)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  self.parser.feed(markup)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  self.parser.feed(markup)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  self.parser.feed(markup)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprec

current docket text table size/shape: (3244, 4)
docket text 0
COMPLAINT against Cardiogenics Holdings, Inc. filing fee $ 400, receipt number 0207-8445206 Was the Disclosure Statement on Civil Cover Sheet completed -YES,, filed by LG Capital Funding, LLC. (Steinmetz, Michael) (Additional attachment(s) added on 3/11/2016: # 1 Civil Cover Sheet, # 2 Proposed Summons) (Bowens, Priscilla). (Entered: 03/10/2016) 

docket text 1
Case assigned to Judge Ann M Donnelly and Magistrate Judge Vera M. Scanlon. Please download and review the Individual Practices of the assigned Judges, located on our website. Attorneys are responsible for providing courtesy copies to judges where their Individual Practices require such. (Bowens, Priscilla) (Entered: 03/11/2016) 

docket text 2
Summons Issued as to Cardiogenics Holdings, Inc.. (Bowens, Priscilla) (Entered: 03/11/2016) 

docket text 3
NOTICE - emailed attorney regarding missing second page of the civil cover sheet. (Bowens, Priscilla) (Entered: 03/11/2

In [143]:
%%time
path_to_model = r'C:\Users\inves\AppData\Local\Programs\Python\Python35\Lib\site-packages\nltk\stanford-ner-2018-02-27\classifiers\english.all.3class.distsim.crf.ser.gz'
path_to_jar = r'C:\Users\inves\AppData\Local\Programs\Python\Python35\Lib\site-packages\nltk\stanford-ner-2018-02-27\stanford-ner.jar'
tagger = StanfordNERTagger(path_to_model, path_to_jar = path_to_jar)

output = []
#length = 100 
length = len(docket_original)
for i in range(length):
    org_str = []
    name_str = []
    stripped_str1 = []
    stripped_str2 = []
    tokens = nltk.tokenize.word_tokenize(docket_original[i])
    for label in tagger.tag(tokens):
        #print(label)
        if label[1] == 'ORGANIZATION':
            org_str.append(label[0])
            stripped_str1.append('-ORG-')
        elif label[1] == 'PERSON':
            name_str.append(label[0])
            stripped_str1.append('-NAME-')
        else:
            stripped_str1.append(label[0])
            stripped_str2.append(label[0])
    
    output.append([docket_original[i],
                   ' '.join(org_str),
                   ' '.join(name_str),
                   ' '.join(stripped_str1),
                   ' '.join(stripped_str2)])
    
new_df = pd.DataFrame(output, columns = ['Original Text', 'Org Part', 'Name Part', 'Stripped Txt', 'RTW Txt'])

Wall time: 1h 36min 51s


In [144]:
print(new_df.head())
docket_text_list = list(new_df['RTW Txt'])

                                       Original Text  \
0  COMPLAINT against Cardiogenics Holdings, Inc. ...   
1  Case assigned to Judge Ann M Donnelly and Magi...   
2  Summons Issued as to Cardiogenics Holdings, In...   
3  NOTICE - emailed attorney regarding missing se...   
4  In accordance with Rule 73 of the Federal Rule...   

                                          Org Part  \
0  Cardiogenics Holdings , Inc. LG Capital Funding   
1      Individual Practices of the assigned Judges   
2                            Cardiogenics Holdings   
3                                                    
4                                                    

                                         Name Part  \
0             ( Steinmetz Michael Bowens Priscilla   
1  Ann M Donnelly Vera M. Scanlon Bowens Priscilla   
2                                 Bowens Priscilla   
3                                 Bowens Priscilla   
4                                 Bowens Priscilla   

             

In [145]:
def text_preprocess1(text):
    text = text.replace('-', '')
    text = text.replace('(', ' ')
    text = text.replace(')', ' ')
    text = text.replace('(s)', 's')
    text = text.replace('*', '')
    return text

def text_preprocess2(text):
    text = text.replace('.', '')
    return text

def remove_stop(sentence):
    output = []
    for word in sentence.split():
        if word not in set(stopwords.words('english')):
            output.append(word)
    return ' '.join(output)

keywords = pd.read_csv('docket_texts/keywords.csv', header = None)
keywords.columns = ['keywords']
keyword_list = list(keywords['keywords'])

In [146]:
docket_text_list = [text_preprocess1(sentence).lower() for sentence in docket_text_list]
print(docket_text_list[1])
print(len(docket_text_list))

case assigned to judge and magistrate judge . please download and review the , located on our website . attorneys are responsible for providing courtesy copies to judges where their individual practices require such .   ,     entered : 03112016  
3244


In [147]:
class Splitter(object):

    def __init__(self):
        self.splitter = nltk.data.load('tokenizers/punkt/english.pickle')
        self.tokenizer = nltk.tokenize.TreebankWordTokenizer()

    def split(self,text):

        # split into single sentence
        sentences = self.splitter.tokenize(text)
        # tokenization in each sentences
        tokens = [self.tokenizer.tokenize(remove_stop(sent)) for sent in sentences]
        return tokens


class LemmatizationWithPOSTagger(object):
    def __init__(self):
        pass
    def get_wordnet_pos(self,treebank_tag):
        """
        return WORDNET POS compliance to WORDENT lemmatization (a,n,r,v) 
        """
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            # As default pos in lemmatization is Noun
            return wordnet.NOUN

    def pos_tag(self,tokens):
        # find the pos tagginf for each tokens [('What', 'WP'), ('can', 'MD'), ('I', 'PRP') ....
        pos_tokens = [nltk.pos_tag(token) for token in tokens]

        # lemmatization using pos tagg   
        # convert into feature set of [('What', 'What', ['WP']), ('can', 'can', ['MD']), ... ie [original WORD, Lemmatized word, POS tag]
        pos_tokens = [ [(word, lemmatizer.lemmatize(word,self.get_wordnet_pos(pos_tag)), [pos_tag]) for (word,pos_tag) in pos] for pos in pos_tokens]
        return pos_tokens

In [148]:
%%time
lemmatizer = WordNetLemmatizer()
splitter = Splitter()
lemmatization_using_pos_tagger = LemmatizationWithPOSTagger()

lemma_docket_text_list = []
for docket_text in docket_text_list:
    #step 1 split document into sentence followed by tokenization
    tokens = splitter.split(docket_text)

    #step 2 lemmatization using pos tagger 
    lemma_pos_token = lemmatization_using_pos_tagger.pos_tag(tokens)
    lemma_docket_text_list.append(lemma_pos_token)

Wall time: 2min 4s


In [149]:
print(len(lemma_docket_text_list)) #docket text document level
print(len(lemma_docket_text_list[0])) #docket text sentence level
print(len(lemma_docket_text_list[0][0])) #docket text word level
print(lemma_docket_text_list[0][0][0]) #docket text token level
print(lemma_docket_text_list[0][0][0][0]) #docket text tuple level

3244
3
22
('complaint', 'complaint', ['NN'])
complaint


In [150]:
#lets do a collection of what we have
collection = {}
for lemma_pos_token in lemma_docket_text_list:
    for sentence in lemma_pos_token:
        for token in sentence:
            #print(token[2][0])
            if token[2][0] not in list(collection.keys()):
                collection[token[2][0]] = []
                collection[token[2][0]].append(token[1])
            else:
                if token[1] not in collection[token[2][0]]:
                    collection[token[2][0]].append(token[1])

In [153]:
pd.DataFrame(dict([ (k, pd.Series(v)) for k, v in collection.items()])).to_csv('NLP_pos.csv', index = False)

In [175]:
%%time
remove_pos = ["``", "NNPS", "NNP", "CD", '#', '$', "''", ",", "0", ":"]
#rebuild corpus
docket_texts_output = [] #ultimate output after cleaning

for lemma_pos_token in lemma_docket_text_list:
    docket_text_output = [] 
    for sentence in lemma_pos_token:
        sentence_output = []
        for token in sentence:
            if token[2][0] not in remove_pos: #if the pos is not in the remove_pos list
                sentence_output.append(token[1]) #append the the sentence
        docket_text_output.append(' '.join(sentence_output))
    docket_texts_output.append(docket_text_output)
print(docket_texts_output[:10])

[['complaint file fee receipt number disclosure statement civil cover sheet complete yes file llc .', 'additional attachment add civil cover sheet proposed summons .', 'enter'], ['case assign judge magistrate judge .', 'please download review locate website .', 'attorney responsible provide courtesy copy judge individual practice require .', 'enter'], ['summons issue inc.. enter'], ['notice email attorney regard miss second page civil cover sheet .', 'enter'], ['accordance rule federal rule civil procedure local rule party notify party consent united state magistrate judge court available conduct proceeding civil action include jury nonjury trial order entry final judgment .', 'attach notice blank copy consent form fill sign file electronically party wish consent .', 'form may also access follow link http www.uscourts.govuscourtsformsandfeesformsao085.pdf .', 'may withhold consent without adverse substantive consequence .', 'return file consent unless party sign consent .', 'enter'], [

In [179]:
new_df['POS treat'] = pd.Series(docket_texts_output)
new_df.head()

,Original Text,Org Part,Name Part,Stripped Txt,RTW Txt,POS treat
0,"COMPLAINT against Cardiogenics Holdings, Inc. ...","Cardiogenics Holdings , Inc. LG Capital Funding",( Steinmetz Michael Bowens Priscilla,COMPLAINT against -ORG- -ORG- -ORG- -ORG- fili...,"COMPLAINT against filing fee $ 400 , receipt n...",[complaint file fee receipt number disclosure ...
1,Case assigned to Judge Ann M Donnelly and Magi...,Individual Practices of the assigned Judges,Ann M Donnelly Vera M. Scanlon Bowens Priscilla,Case assigned to Judge -NAME- -NAME- -NAME- an...,Case assigned to Judge and Magistrate Judge . ...,"[case assign judge magistrate judge ., please ..."
2,"Summons Issued as to Cardiogenics Holdings, In...",Cardiogenics Holdings,Bowens Priscilla,"Summons Issued as to -ORG- -ORG- , Inc.. ( -NA...","Summons Issued as to , Inc.. ( , ) ( Entered :...",[summons issue inc.. enter]
3,NOTICE - emailed attorney regarding missing se...,,Bowens Priscilla,NOTICE - emailed attorney regarding missing se...,NOTICE - emailed attorney regarding missing se...,[notice email attorney regard miss second page...
4,In accordance with Rule 73 of the Federal Rule...,,Bowens Priscilla,In accordance with Rule 73 of the Federal Rule...,In accordance with Rule 73 of the Federal Rule...,[accordance rule federal rule civil procedure ...


In [180]:
unigram_sentences_filepath = 'docket_texts/unigram_nltk_noorgnoname.txt'

In [181]:
%%time
# turn the lemmatized corpus into unigram sentences
with codecs.open(unigram_sentences_filepath, 'w', encoding = 'utf_8') as f:
    for docket_text in docket_texts_output:
        for sentence in docket_text:
            f.write(sentence + '\n')

Wall time: 22.1 ms


In [183]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

In [184]:
#let's do some comparision between the original text and unigram sentences, shouldn't be that different.
print('original text:')
print(new_df['POS treat'].iloc[0])
#print(df['Docket Text'].iloc[1])

print('\nunigram_sentence:')
for unigram_sentence in it.islice(unigram_sentences, 0, 10):
    print(' '.join(unigram_sentence))
    print('')

original text:
['complaint file fee receipt number disclosure statement civil cover sheet complete yes file llc .', 'additional attachment add civil cover sheet proposed summons .', 'enter']

unigram_sentence:
complaint file fee receipt number disclosure statement civil cover sheet complete yes file llc .

additional attachment add civil cover sheet proposed summons .

enter

case assign judge magistrate judge .

please download review locate website .

attorney responsible provide courtesy copy judge individual practice require .

enter

summons issue inc.. enter

notice email attorney regard miss second page civil cover sheet .

enter



In [185]:
bigram_model_filepath = 'docket_texts/bigram_model_noorgnoname' 

In [187]:
%%time

# store our bigram model
bigram_model = Phrases(unigram_sentences)
bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk if we don't want to run this again
#bigram_model = Phrases.load(bigram_model_filepath)

Wall time: 206 ms


In [188]:
bigram_sentences_filepath = 'docket_texts/bigram_sentences_noorgnoname.txt'

In [189]:
%%time

# apply the bigram model, and write it to file
with codecs.open(bigram_sentences_filepath, 'w', encoding = 'utf_8') as f:
    for unigram_sentence in unigram_sentences:
        bigram_sentence = ' '.join(bigram_model[unigram_sentence])
        f.write(bigram_sentence + '\n')

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Wall time: 567 ms


In [190]:
bigram_sentences = LineSentence(bigram_sentences_filepath)
print('unigram length = {}, bigram length = {}'.format(len(list(unigram_sentences)), len(list(bigram_sentences))))

unigram length = 11502, bigram length = 11502


In [226]:
#original v. unigram v. bigram. Some phrases should be combined already
start = 0
finish = 10
print('original text:')
print(new_df['POS treat'].iloc[0])
print(new_df['POS treat'].iloc[1])

print('\nunigram sentence:')
for unigram_sentence in it.islice(unigram_sentences, 0, 10):
    print(' '.join(unigram_sentence))
print('\nbigram sentence:')
for bigram_sentence in it.islice(bigram_sentences, start, finish):
    print(' '.join(bigram_sentence))

original text:
['complaint file fee receipt number disclosure statement civil cover sheet complete yes file llc .', 'additional attachment add civil cover sheet proposed summons .', 'enter']
['case assign judge magistrate judge .', 'please download review locate website .', 'attorney responsible provide courtesy copy judge individual practice require .', 'enter']

unigram sentence:
complaint file fee receipt number disclosure statement civil cover sheet complete yes file llc .
additional attachment add civil cover sheet proposed summons .
enter
case assign judge magistrate judge .
please download review locate website .
attorney responsible provide courtesy copy judge individual practice require .
enter
summons issue inc.. enter
notice email attorney regard miss second page civil cover sheet .
enter

bigram sentence:
complaint file fee_receipt number disclosure_statement civil_cover sheet complete_yes file llc .
additional attachment_add civil_cover sheet proposed summons .
enter
case 

In [227]:
trigram_model_filepath = 'docket_texts/trigram_model_nonamenoorg'

In [260]:
%%time

# again, using Phrases to attach more words to phrases already formed
trigram_model = Phrases(bigram_sentences)
trigram_model.save(trigram_model_filepath)

# load the finished model from disk
#trigram_model = Phrases.load(trigram_model_filepath)

Wall time: 213 ms


In [261]:
trigram_sentences_filepath = 'docket_texts/trigram_sentences_nonamenoorg.txt'

In [263]:
%%time

with codecs.open(trigram_sentences_filepath, 'w', encoding = 'utf_8') as f:
    for bigram_sentence in bigram_sentences:
        #print('Bi', bigram_sentence)
        trigram_sentence = ' '.join(trigram_model[bigram_sentence])
        #print('Tri', trigram_sentence)
        f.write(trigram_sentence + '\n')

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Wall time: 455 ms


In [264]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [265]:
start = 0
finish = 15
print('original text:')
print(new_df['POS treat'].iloc[0],'\n')
print(new_df['POS treat'].iloc[1],'\n')
print(new_df['POS treat'].iloc[2],'\n')
print(new_df['POS treat'].iloc[3],'\n')

print('\nUNIGRAM Sentence:')
for unigram_sentence in it.islice(unigram_sentences, start, finish):
    print(' '.join(unigram_sentence))
print('\nBIGRAM Sentence:')
for bigram_sentence in it.islice(bigram_sentences, start, finish):
    print(' '.join(bigram_sentence))
print('\nTRIGRAM Sentence:')
for trigram_sentence in it.islice(trigram_sentences, start, finish):
    print(' '.join(trigram_sentence))

original text:
['complaint file fee receipt number disclosure statement civil cover sheet complete yes file llc .', 'additional attachment add civil cover sheet proposed summons .', 'enter'] 

['case assign judge magistrate judge .', 'please download review locate website .', 'attorney responsible provide courtesy copy judge individual practice require .', 'enter'] 

['summons issue inc.. enter'] 

['notice email attorney regard miss second page civil cover sheet .', 'enter'] 


UNIGRAM Sentence:
complaint file fee receipt number disclosure statement civil cover sheet complete yes file llc .
additional attachment add civil cover sheet proposed summons .
enter
case assign judge magistrate judge .
please download review locate website .
attorney responsible provide courtesy copy judge individual practice require .
enter
summons issue inc.. enter
notice email attorney regard miss second page civil cover sheet .
enter
accordance rule federal rule civil procedure local rule party notify par

In [284]:
def trigram_transform(texts):
    trigram_output = []
    #print(texts)
    
    for sentence in texts:
        unigram_review = []
        for word in sentence.split():
            unigram_review.append(word)
    
        #print('Uni: ', unigram_review)
        bigram_review = bigram_model[unigram_review]
        #print('Bi: ', bigram_review)
        trigram_review = trigram_model[bigram_review]
        #print('Tri: ', trigram_review)
        trigram_output.append(' '.join(trigram_review))
    return trigram_output

In [287]:
new_df['Phrase Model'] = new_df['POS treat'].apply(trigram_transform)

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [289]:
new_df['Phrase Model'].iloc[0]

['complaint file fee_receipt_number disclosure_statement_civil_cover sheet_complete_yes file llc .',
 'additional_attachment_add civil_cover_sheet proposed summons .',
 'enter']

In [290]:
#write trigram to file
trigram_dockets_filepath = 'docket_texts/trigram_transformed_dockets_noorgnoname.txt'

In [299]:
with codecs.open(trigram_dockets_filepath, 'w', encoding= 'utf_8') as f:
    for i in range(len(new_df['Phrase Model'])):
        f.write(' '.join(new_df['Phrase Model'][i]) + '\n')

In [300]:
trigram_dictionary_filepath = 'docket_texts/trigram_dict_noorgnoname.dict'

In [303]:
%%time

#some dictionary hyperparameters:
no_below = 10 #reference is 10
no_above = 0.4 #reference is 0.4

trigram_reviews = LineSentence(trigram_dockets_filepath)

# learn the dictionary by iterating over all of the reviews
trigram_dictionary = Dictionary(trigram_reviews)

# filter tokens that are very rare otrigram_reviewsr too common from
# the dictionary (filter_extremes) and reassign integer ids (compactify)
trigram_dictionary.filter_extremes(no_below = no_below, no_above = no_above) #this step is questionable. May need to change the parameters
trigram_dictionary.compactify()

trigram_dictionary.save(trigram_dictionary_filepath)
    
# load the finished dictionary from disk
#trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

Wall time: 104 ms


In [305]:
trigram_bow_filepath = 'docket_texts/trigram_bow_corpus_noorgnoname.mm'

In [306]:
def trigram_bow_generator(filepath):
    """
    generator function to read reviews from a file
    and yield a bag-of-words representation
    """
    
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [307]:
%%time

# generate bag-of-words representations for
# all reviews and save them as a matrix
MmCorpus.serialize(trigram_bow_filepath, trigram_bow_generator(trigram_sentences_filepath))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

Wall time: 212 ms


In [337]:
def explore_topic(model, topic_number, topn = 10):
    topics = []
    print('{:20} {}'.format('term', 'frequency') + '\n')
    for term, frequency in model.show_topic(topic_number, topn = topn):
        print('{:20} {:.3f}'.format(term, round(frequency, 3)))
        topics.append((term, round(frequency, 3)))
    return topics

In [342]:
def topic_modeling_pipeline(num_topics, model_file_path, trigram_bow_corpus, trigram_dictionary, export = False):

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')

        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        lda = LdaMulticore(trigram_bow_corpus, num_topics = num_topics, id2word = trigram_dictionary, workers = 4)

    lda.save(model_file_path)
    
    topic_dict = {}
    for i in range(num_topics):
        print("\n Topic {}'s make-up:".format(i + 1))
        topic_dict[i] = explore_topic(lda, topic_number = i)
    
    if export:
        pd.DataFrame(topic_dict).to_csv(model_file_path + 'topics.csv')
    
    LDAvis_data_filepath = model_file_path + '_ldavis'
    
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus, trigram_dictionary)

    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
        
    with open(LDAvis_data_filepath, 'rb') as f:
        LDAvis_prepared = pickle.load(f)
        
    return LDAvis_prepared

In [343]:
lda2_model_filepath = 'docket_texts/lda_model_noorgnomodel_2'
LDAvis_prepared = topic_modeling_pipeline(2, lda2_model_filepath, trigram_bow_corpus, trigram_dictionary, True)
pyLDAvis.display(LDAvis_prepared)


 Topic 1's make-up:
term                 frequency

motion               0.066
order                0.031
notice               0.022
's                   0.020
party                0.013
sign_judge           0.013
court                0.011
case                 0.010
request              0.009
stipulation          0.009

 Topic 2's make-up:
term                 frequency

exhibit              0.122
order                0.049
motion               0.042
defendant            0.014
shall                0.012
letter               0.010
attachment           0.009
's                   0.009
dismiss              0.008
date                 0.008
                      0                    1
0       (motion, 0.066)     (exhibit, 0.122)
1        (order, 0.031)       (order, 0.049)
2       (notice, 0.022)      (motion, 0.042)
3            ('s, 0.02)   (defendant, 0.014)
4        (party, 0.013)       (shall, 0.012)
5   (sign_judge, 0.013)       (letter, 0.01)
6        (court, 0.011)  (attachment, 0

In [330]:
lda3_model_filepath = 'docket_texts/lda_model_noorgnomodel_3'
LDAvis_prepared = topic_modeling_pipeline(3, lda3_model_filepath, trigram_bow_corpus, trigram_dictionary)
pyLDAvis.display(LDAvis_prepared)


 Topic 1's make-up:
term                 frequency

motion               0.066
sign_judge           0.019
's                   0.019
order                0.019
notice               0.017
ex                   0.012
attachment           0.012
defendant            0.012
exhibit              0.011
date                 0.011

 Topic 2's make-up:
term                 frequency

motion               0.065
exhibit              0.052
order                0.050
notice               0.015
judge                0.014
stipulation          0.012
shall                0.012
court                0.011
plaintiff            0.011
inc..                0.011

 Topic 3's make-up:
term                 frequency

exhibit              0.131
order                0.048
motion               0.030
defendant            0.016
's                   0.014
party                0.014
case                 0.012
b                    0.009
shall                0.009
magistrate_judge     0.009


In [331]:
lda5_model_filepath = 'docket_texts/lda_model_noorgnomodel_5'
LDAvis_prepared = topic_modeling_pipeline(5, lda5_model_filepath, trigram_bow_corpus, trigram_dictionary)
pyLDAvis.display(LDAvis_prepared)


 Topic 1's make-up:
term                 frequency

motion               0.050
exhibit              0.038
order                0.023
's                   0.020
shall                0.015
dismiss              0.014
letter               0.014
ex                   0.013
notice               0.012
schedule             0.011

 Topic 2's make-up:
term                 frequency

exhibit              0.255
order                0.050
judge                0.015
shall                0.015
attachment           0.013
magistrate_judge     0.013
case                 0.012
minute_entry_proceeding_hold 0.011
defendant            0.010
request              0.010

 Topic 3's make-up:
term                 frequency

motion               0.070
inc..                0.029
notice               0.025
's                   0.022
serve_answer_due     0.014
dismiss              0.014
sign_judge_iii       0.013
defendant            0.012
mro                  0.011
summary_judgment     0.009

 Topic 4's make-up:
te

In [332]:
lda10_model_filepath = 'docket_texts/lda_model_noorgnomodel_10'
LDAvis_prepared = topic_modeling_pipeline(10, lda10_model_filepath, trigram_bow_corpus, trigram_dictionary)
pyLDAvis.display(LDAvis_prepared)


 Topic 1's make-up:
term                 frequency

motion               0.033
order                0.031
magistrate_judge     0.020
sign_judge           0.017
judge                0.015
due                  0.015
case                 0.014
request              0.014
discovery            0.013
's                   0.012

 Topic 2's make-up:
term                 frequency

order                0.077
motion               0.070
party                0.017
grant                0.017
defendant            0.017
plaintiff            0.015
sign_judge           0.013
letter               0.011
ex                   0.011
dismiss              0.011

 Topic 3's make-up:
term                 frequency

motion               0.104
order                0.041
stipulation          0.024
defendant            0.023
's                   0.018
memorandum_law_support 0.016
court                0.016
exhibit              0.015
dismiss              0.014
shall                0.013

 Topic 4's make-up:
term    